In [11]:
import awscli
import selenium
import boto3
import pandas as pd
import time
from selenium import webdriver
import csv
import pandas as pd
import numpy as np
import logging
from botocore.exceptions import ClientError

####SCRAPE THE WEBSITE######
###call the webdriver
browser = webdriver.Chrome("C:\Windows\chromedriver.exe")

#enter the url path that needs to be accessed by webdriver
#browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities_action.jsp')
browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities_action.jsp?orgName=a&d-49653-p=1&city=&searchType=contains&reg1=&project=Charities&reg3=&reg2=&ein=&orgId=&num1=&state=none&regType=ALL&num2=')
#browser.get('https://www.charitiesnys.com/RegistrySearch/search_charities_action.jsp')

#identify xpath of location to select element
inputElement = browser.find_element_by_xpath("/html/body/div[2]/div/table/tbody/tr/td[3]/div/div/table[2]/tbody/tr[1]")
#inputElement.send_keys('0')
inputElement1 = browser.find_element_by_xpath("/html/body/div[2]/div/table/tbody/tr/td[3]/div/div/table[2]/tbody/tr[2]").click()


#identify xpath of location to select element
#inputElement = browser.find_element_by_xpath("/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[2]/td[2]/input[1]")
#inputElement.send_keys('0')
#inputElement1 = browser.find_element_by_xpath("/html/body/div/div[2]/div/table/tbody/tr/td[2]/div/div/font/font/font/font/font/font/table/tbody/tr[4]/td/form/table/tbody/tr[10]/td/input[1]").click()

#identify the table to scrape
table = browser.find_element_by_css_selector('table.Bordered')

print(table)

<selenium.webdriver.remote.webelement.WebElement (session="43887e53972fc15b526aeb78945e5692", element="766f5eec-9642-4c88-813f-1f340ec68940")>


In [12]:
#####CREATE DATE FRAME#####
#create empty dataframe
df =[]

#loop through dataframe to export table
for row in table.find_elements_by_css_selector('tr'):
      cols = df.append([cell.text for cell in row.find_elements_by_css_selector('td')])


#update dataframe with header 
df = pd.DataFrame(df, columns = ["Organization Name", "NY Reg #", "EIN" ,"Registrant Type","City","State"])
display(df) #let's have a look at the data before creating the CSV file and loading it into s3

,Organization Name,NY Reg #,EIN,Registrant Type,City,State
0,None,None,None,None,None,None
1,.Congregation BSZ Inc.,48-32-12,862626128,NFP,MONSEY,NY
2,".NO - Art From Elsewhere, Inc.",44-18-72,274184914,NFP,BROOKLYN,
3,"""THEY ARE HAITIAN"" FUND, INC.",20-63-46,300170128,NFP,HUDSON,NY
4,"""Tiwimuta"" Inc.",42-21-31,262541693,NFP,BROOKLYN,NY
5,"""We Are Many"" Foundation Inc.",43-50-81,451151323,NFP,HAUPPAUGE,NY
6,(ASMA) American Syrian Multicultural Associati...,42-84-63,273130182,NFP,BROOKLYN,NY
7,(D.A.S.H. Ministries Inc.) Daughters Achieving...,44-00-22,461955332,NFP,BRONX,NY
8,"(Rev.) Dwyer, John F. Estate a/k/a John F. Dwyer",48-15-01,NO DATA AVAILABLE,Estates,NO DATA AVAILABLE,
9,#Cancerland,46-36-65,472293681,NFP,YONKERS,NY


In [35]:
# prepare csv file name   
pathname = 's3:/m10assignment-bucket/'#specify location of s3:/{my-bucket}/
filename= 'charities_bureau_scrape_' #name of your group
datetime = time.strftime("%Y%m%d%H%M%S") #timestamp
filenames3 = "%s%s%s.csv"%(pathname,filename,datetime) #name of the filepath and csv file
names = filenames3[25:]

df.to_csv('C:/Users/Bernie/charities.csv')
#load file into s3. Pandas actually leverages boto to connect to s3 and can push the file directly into an s3 bucket
#df.to_csv(filenames3, header=True, line_terminator='\n') 

s3 = boto3.resource('s3')
s3.meta.client.upload_file('C:/Users/Bernie/charities.csv', 'm10assignment-bucket', names)